In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_teddynote import logging
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [6]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()
# llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D664E32690>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D667583650>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [21]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:8b")

# from_template()메소드를 사용하여 PromptTemplate객체 생성
- 치환할 변수를 `{변수}` 로 묶어서 템플릿을 정의 - 중괄호 안에 있는것이 변수이다.

In [23]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [24]:
# format을 통해서 중간에 변수를 임의로 채워서 확인할 수 있다 .
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [25]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [28]:
chain = prompt | llm
print(chain.invoke({"country":"대한민국"}))
print(chain.invoke({"country":"대한민국"}).content)

content='😊\n\nThe capital of South Korea is Seoul (,).' response_metadata={'model': 'llama3:8b', 'created_at': '2024-07-04T16:22:03.8948344Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 533675000, 'load_duration': 3530900, 'prompt_eval_duration': 257069000, 'eval_count': 14, 'eval_duration': 271293000} id='run-124747e7-4af3-4a3a-a633-c0d019c57534-0'
😊

The capital of South Korea is Seoul (, Hangugeo:). It is the largest city in Korea and the country's economic, cultural, and political hub.


# PromptTemplate객체 생성과 동시에 prompt생성

In [30]:
template  = "{country}의 수도는 어디인가요?"

teomplate = PromptTemplate(
    template=template,
    input_variables = ["country"],
)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [31]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [32]:
template = "{country1}과 {country2}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=['country1'],
    partial_variables={
        "country2": "미국"
    }
)
# country2는 미국으로 채워진 상태로 들어간다.
prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 어디인가요?')

In [33]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 어디인가요?'

In [34]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], template='{country1}과 {country2}의 수도는 어디인가요?')

In [39]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 어디인가요?')

In [40]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 어디인가요?'

In [41]:
chain = prompt_partial | llm

In [42]:
chain.invoke("대한민국").content

'😊\n\nThe capital of South Korea (대한민국) is Seoul (, Seoul), which is located in the northwest part of the country.\n\nAnd the capital of Canada is Ottawa (, Ottawa), which is located in the province of Ontario. 🇨🇦'

In [43]:
chain.invoke({"country1":"대한민국", "country2":"호주"}).content

'😊\n\nThe capital of South Korea (대한민국) is Seoul (, officially known as Seoul Special Metropolitan City).\n\nAnd the capital of Australia (호주) is Canberra ().\n\nSo, they are two separate countries with their own capitals! 🇰🇷🇦🇺'

# Partial_variables: 부분변수 채움

In [47]:
from datetime import datetime

datetime.now().strftime("%B %d")

'July 05'

In [48]:
def get_today():
    return datetime.now().strftime("%bB %d")

In [49]:
prompt = PromptTemplate(
    template = "오늘의 날짜는 {today}입니다. 오늘이 생일인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables = ["n"],
    partial_variables={
        "today":get_today
    },
)
prompt

PromptTemplate(input_variables=['n'], partial_variables={'today': <function get_today at 0x000001D66B3C7C40>}, template='오늘의 날짜는 {today}입니다. 오늘이 생일인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [50]:
prompt.format(n=3)

'오늘의 날짜는 JulB 05입니다. 오늘이 생일인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [56]:
chain = prompt | llm

In [57]:
print(chain.invoke(3).content)

🎉

Based on the date you provided, July 5th (Jul 05), here are three people who share their birthday with you:

1. **Marilyn Monroe** - June 1, 1926
2. **Lucille Ball** - August 6, 1911
3. **Kirk Douglas** - December 9, 1916

Note: These individuals may not be alive today, but they share the same birthday as you! 🎂


In [58]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

Happy birthday to those born on January 2nd!

Here are three notable individuals born on this day:

1. **Pierre Trudeau** (January 2, 1918) - Canadian politician and lawyer, father of current Canadian Prime Minister Justin Trudeau.
2. **James Levine** (January 2, 1943) - American conductor, music director of the Chicago Symphony Orchestra from 1986 to 2014.
3. **Sylvain Chomet** (January 2, 1965) - French animator and filmmaker, known for his work on films like "The Triplets of Belleville" and "Delicatessen".

Hope this birthday shoutout brings a smile to your face! 🎉


# 파일로부터 template읽어오기

In [67]:
from langchain_core.prompts import load_prompt

#load_prompt("prompts/fruit_color.yaml", encoding="cp949")
prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")

In [68]:
prompt

PromptTemplate(input_variables=['fruit'], template='{fruit}의 색깔이 뭐야?')

In [70]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [71]:
prompt2 = load_prompt("prompts/capital.yaml", encoding="utf-8")
prompt2

PromptTemplate(input_variables=['country'], template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')

In [73]:
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [74]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:8b")

In [75]:
from langchain_core.output_parsers import StrOutputParser

In [77]:
chain = prompt2 | llm | StrOutputParser()

In [79]:
print(chain.invoke({"country": "대한민국"}))

🇰🇷

대한민국의 수도는 서울(Segyeol)입니다.

**1. 면적**
서울시의 면적은 605.25km²입니다.

**2. 인구**
2020년 기준으로, 서울시의 인구는 약 10,221,000명입니다.

**3. 역사적 장소**
서울은 대한민국의 역사를 자랑하는 도시입니다. 고조선 시대부터 조선왕朝, 대한제국, 일제 강점기까지 다양한 역사를 갖추고 있습니다. 대표적인 역사적 장소로는 경복궁, 창덕궁, 덕수궁 등이 있습니다.

**4. 특산품**
서울은 곡물과 과일 등의 농산물을 생산합니다. 특히, 서울의 특산품으로는 감자, 밤, 홍삼, 등심 등이 있습니다.

이러한 특징으로 인해, 서울은 대한민국의 수도로써 중요한 역할을 수행하고 있습니다.


In [80]:
from langchain_teddynote.messages import stream_response

In [81]:
answer = chain.stream({"country": "대한민국"})

In [83]:
stream_response(answer)

Here's the information about the capital of South Korea, Seoul:

**서울특별시 (Seoul Special City)**

**1. 면적 (Area)**
: 605.25 km²

**2. 인구 (Population)**
: approximately 10 million people (as of 2020)

**3. 역사적 장소 (Historic Places)**
: Gyeongbokgung Palace, Changdeokgung Palace, Bukchon Hanok Village, Namsan Tower (Seoul Tower), and the National Folk Museum are some of the many historic places to visit in Seoul.

**4. 특산품 (Special Products)**
: Kimchi, doenjang (fermented soybean paste), gochujang (Korean chili paste), and handmade crafts like pottery and woodcarvings are some of the special products that can be found in Seoul.

Note: The information is accurate as of 2020, but please note that population figures may have changed since then.

In [84]:
for token in answer:
    print(token.content, end="", flush=True)

In [85]:
def merge_docs(retrieved_docs):
    return "\n\n".join([d.content for d in retrieved_docs])

In [ ]:
# ChatPromptTemplate